In [2]:
import torch, types

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 6)

In [1]:
#현재깔린게 2.2임.
import torch, triton
print(torch.__version__)      # 2.2.2  (CUDA 12.1)
print(triton.__version__)     # 2.2.0  ← OK
from triton.runtime.jit import get_cuda_stream     # 오류 없으면 정상


2.2.2
2.2.0


- unsloth_zoo가 PyTorch ≥ 2.3에 새로 추가된 torch.amp.is_autocast_available() API를 호출합니다.
- 현재 SageMaker 노트북의 PyTorch 2.2.2 에는 torch.amp.is_autocast_available() 함수가 없어 AttributeError가 발생


## Unsloth

- `Unsloth`는 Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes 등을 지원합니다. 그리고 Yi, Qwen([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, 모든 Llama, Mistral 파생 아키텍처도 지원합니다.

- `Unsloth`는 16비트 LoRA 또는 4비트 QLoRA를 지원합니다. 둘 다 2배 빠릅니다.

- `max_seq_length`는 [kaiokendev의](https://kaiokendev.github.io/til) 방법을 통해 자동으로 RoPE 스케일링을 하기 때문에 어떤 값으로도 설정할 수 있습니다.

**새로운 소식**!

- [PR 26037](https://github.com/huggingface/transformers/pull/26037)을 통해, 우리는 4비트 모델을 **4배 빠르게** 다운로드할 수 있는 기능을 지원합니다! [Unsloth Repository](https://huggingface.co/unsloth)에는 Llama, Mistral 4비트 모델이 있습니다.


`FastLanguageModel.from_pretrained` 함수를 사용하여 사전 훈련된 언어 모델을 로드하는 과정을 설명합니다.

- 최대 시퀀스 길이(`max_seq_length`)를 설정하여 모델이 처리할 수 있는 입력 데이터의 길이를 지정합니다.
- 데이터 타입(`dtype`)은 자동 감지되거나, 특정 하드웨어에 최적화된 형식(`Float16`, `Bfloat16`)으로 설정할 수 있습니다.
- 4비트 양자화(`load_in_4bit`) 옵션을 사용하여 메모리 사용량을 줄일 수 있으며, 이는 선택적입니다.
- 사전 정의된 4비트 양자화 모델 목록(`fourbit_models`)에서 선택하여 다운로드 시간을 단축하고 메모리 부족 문제를 방지할 수 있습니다.
- `FastLanguageModel.from_pretrained` 함수를 통해 모델과 토크나이저를 로드하며, 이때 모델 이름(`model_name`), 최대 시퀀스 길이, 데이터 타입, 4비트 로딩 여부를 매개변수로 전달합니다.
- 선택적으로, 특정 게이트 모델을 사용할 경우 토큰(`token`)을 제공할 수 있습니다.


In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # 모델 이름을 설정합니다.
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A10G. Num GPUs = 1. Max memory: 21.975 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.2.2. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 2.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

beomi/Llama-3-Open-Ko-8B-Instruct-preview does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


이제 LoRA 어댑터를 추가하여 모든 파라미터 중 단 1% ~ 10%의 파라미터만 업데이트하면 됩니다!


FastLanguageModel을 사용하여 특정 모듈에 대한 성능 향상 기법을 적용한 모델을 구성합니다.

- `FastLanguageModel.get_peft_model` 함수를 호출하여 모델을 초기화하고, 성능 향상을 위한 여러 파라미터를 설정합니다.
- `r` 파라미터를 통해 성능 향상 기법의 강도를 조절합니다. 권장 값으로는 8, 16, 32, 64, 128 등이 있습니다.
- `target_modules` 리스트에는 성능 향상을 적용할 모델의 모듈 이름들이 포함됩니다.
- `lora_alpha`와 `lora_dropout`을 설정하여 LoRA(Low-Rank Adaptation) 기법의 세부 파라미터를 조정합니다.
- `bias` 옵션을 통해 모델의 바이어스 사용 여부를 설정할 수 있으며, 최적화를 위해 "none"으로 설정하는 것이 권장됩니다.
- `use_gradient_checkpointing` 옵션을 "unsloth"로 설정하여 VRAM 사용량을 줄이고, 더 큰 배치 크기로 학습할 수 있도록 합니다.
- `use_rslora` 옵션을 통해 Rank Stabilized LoRA를 사용할지 여부를 결정합니다.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=32,  # LoRA 알파 값을 설정합니다.
    lora_dropout=0.05,  # 드롭아웃을 지원합니다.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.5.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
#tokenizer

### 데이터 준비

**[중요]**

- 토큰화된 출력에 **EOS_TOKEN**을 추가하는 것을 잊지 마세요! 그렇지 않으면 무한 생성이 발생할 수 있습니다.

**[참고]**

- 오직 완성된 텍스트만을 학습하고자 한다면, TRL의 문서를 [여기](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)에서 확인하세요.


`load_dataset` 함수를 사용하여 특정 데이터셋을 로드하고, 이를 특정 형식으로 포매팅하는 과정을 설명합니다.

- `load_dataset` 함수로 "teddylee777/QA-Dataset-mini" 데이터셋을 "train" 분할로 로드합니다.
- 데이터셋의 각 예제에 대해 `formatting_prompts_func` 함수를 적용하여 포매팅을 수행합니다.
  - 이 함수는 "instruction"과 "output" 필드를 사용하여 주어진 포맷에 맞게 텍스트를 재구성합니다.
  - 재구성된 텍스트는 `alpaca_prompt` 포맷을 따르며, 각 항목의 끝에는 `EOS_TOKEN`을 추가하여 생성이 종료되도록 합니다.
- 최종적으로, 포매팅된 텍스트는 "text" 키를 가진 딕셔너리 형태로 반환됩니다.
- 이 과정을 통해, AI 모델이 처리하기 적합한 형태로 데이터를 전처리하는 방법을 보여줍니다.


In [5]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# Prompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }




In [7]:
# 로컬 JSONL 파일 로드
dataset = load_dataset('json', data_files='data/train_data.jsonl', split='train')

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 17
})

### 모델 훈련하기

이제 Huggingface TRL의 `SFTTrainer`를 사용해 봅시다!

- 참고 문서: [TRL SFT 문서](https://huggingface.co/docs/trl/sft_trainer)


In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        eval_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/17 [00:00<?, ? examples/s]

- GPU의 현재 메모리 상태를 확인합니다.
- `torch.cuda.get_device_properties(0)`를 사용하여 첫 번째 GPU의 속성을 조회합니다.
- `torch.cuda.max_memory_reserved()`를 통해 현재 예약된 최대 메모리를 GB 단위로 계산합니다.
- GPU의 총 메모리 크기를 GB 단위로 계산합니다.
- GPU 이름과 최대 메모리 크기, 현재 예약된 메모리 크기를 출력합니다.


In [17]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A10G. Max memory = 21.975 GB.
6.939 GB of memory reserved.


In [18]:
#monkey patch 패치 적용
import torch.amp
import contextlib

# is_autocast_available 함수 추가
if not hasattr(torch.amp, 'is_autocast_available'):
    def is_autocast_available(device_type):
        if device_type == 'cuda':
            return True
        elif device_type == 'cpu':
            return hasattr(torch.cpu, 'amp') and hasattr(torch.cpu.amp, 'autocast')
        else:
            return False
    
    torch.amp.is_autocast_available = is_autocast_available
    

In [19]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17 | Num Epochs = 50 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss,Validation Loss
1,1.664900,1.702583
2,1.678700,1.482254
3,1.957900,1.293880
4,1.287900,1.090297
5,1.100000,0.942653
6,0.874600,0.851389
7,0.946300,0.728492
8,0.743900,0.583090
9,0.351900,0.541703
10,0.555800,0.451632


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


PyTorch를 사용하여 모델 훈련 시 메모리 사용량과 훈련 시간을 계산하고 출력하는 코드입니다.

- `torch.cuda.max_memory_reserved()`를 사용하여 훈련 중에 예약된 최대 GPU 메모리를 계산합니다.
- 훈련 시작 시점의 GPU 메모리 사용량과 비교하여 LoRA(Low-Rank Adaptation)를 위해 사용된 추가 메모리 양을 계산합니다.
- 전체 GPU 메모리 대비 사용된 메모리의 비율을 계산합니다.
- 훈련에 소요된 총 시간을 초와 분 단위로 출력합니다.
- 예약된 최대 메모리, LoRA를 위해 사용된 메모리, 그리고 이들이 전체 GPU 메모리 대비 차지하는 비율을 출력합니다.


In [20]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

240.0419 seconds used for training.
4.0 minutes used for training.
Peak reserved memory = 6.941 GB.
Peak reserved memory for training = 0.002 GB.
Peak reserved memory % of max memory = 31.586 %.
Peak reserved memory for training % of max memory = 0.009 %.


### 추론

모델을 실행해 봅시다! 지시사항과 입력값을 변경할 수 있으며, 출력값은 비워두세요!


`TextStreamer`를 사용하여 연속적인 추론을 수행할 수도 있습니다. 이를 통해 전체를 기다리는 대신 토큰별로 생성 결과를 확인할 수 있습니다.


- `FastLanguageModel.for_inference(model)`을 호출하여 모델의 추론 속도를 2배 향상시킵니다.
- `tokenizer`를 사용하여 특정 포맷의 프롬프트를 토큰화하고, 이를 CUDA 기반의 텐서로 변환합니다. 이 과정에서 피보나치 수열을 계속하는 지시문, 입력값, 그리고 출력값을 위한 빈 공간을 포함합니다.
- `TextStreamer` 객체를 `tokenizer`와 함께 초기화하여 텍스트 스트리밍 기능을 활성화합니다.
- `model.generate` 함수를 호출하여 주어진 입력에 대한 텍스트 생성을 수행합니다. 이때, 최대 128개의 새로운 토큰을 생성할 수 있도록 설정하고, `TextStreamer`를 사용하여 결과를 스트리밍합니다.


`StoppingCriteria`와 `StoppingCriteriaList`를 사용하여 특정 토큰에서 생성을 중단하는 방법을 구현합니다.

- `StopOnToken` 클래스는 `StoppingCriteria`를 상속받아, 생성 중 특정 토큰(`stop_token_id`)이 나타나면 생성을 중단하도록 합니다.
- `stop_token` 변수에 중단할 토큰을 문자열로 지정합니다.
- `tokenizer.encode` 메소드를 사용하여 `stop_token`을 해당 언어 모델의 토큰 ID로 변환합니다.
- `StoppingCriteriaList`에 `StopOnToken` 인스턴스를 포함시켜, 생성 과정에서 이를 중단 조건으로 사용합니다.


In [21]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

(예시 1)

In [22]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt.format(
            "전현상은 누구입니까?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
전현상은 누구입니까?

### Response:
전현상은 현재 Amazon Web Services의 Solutions Architect입니다.<|end_of_text|>


In [24]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt.format(
            "금융보안교육센터의 주소?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
금융보안교육센터의 주소?

### Response:
금융보안교육센터의 주소는 서울특별시 영등포구 의사당대로 143 금투센터 8층 입니다.<|end_of_text|>


(예시2)